In [39]:
import pandas as pd
import numpy as np
import re
#import unicode
from functools import reduce
import ast

import unicodedata
from sklearn.preprocessing import MultiLabelBinarizer

In [40]:
pd.set_option("display.max_columns", None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

In [41]:
#modificado por ruta relatiba Gibhud
#dfbarrios= pd.read_excel('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/distr_barrios_codpostal.xlsx')
#dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionSilvia23122024.csv',index_col=False)

#dfpisos= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionAle29122024.csv')

In [42]:
#dfbarrios= pd.read_excel('./data/distr_barrios_codpostal.xlsx')
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')
#dfi= pd.read_csv('./data/dUnionSilvia23122024.csv',index_col=False)
dfi_1= pd.read_csv('./data/dUnionSilvia23122024.csv',index_col=False)
dfiprecios= pd.read_csv('./data/dunionsilviasoloprecios.csv',sep=";",index_col=False)
dfpisos= pd.read_csv('./data/dUnionAle29122024.csv',sep=";",index_col=False)
#utilizo el fichero pisoscom_limpio que parece que aun tenia los valores de m2_constr
dfpcomlimpio= pd.read_csv('../02-datos-limpios/pisoscom_limpio.csv',sep=";",index_col=False)
dfisolometros= pd.read_excel('../02-datos-limpios/Indomiosolometros.xlsx')

In [43]:
dfi_1["title"]

0                                         Ático Calle de Núñez de Balboa 2, Recoletos, Madrid
1       Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid
2                                         Casa plurifamiliar Plaza de Italia, Legazpi, Madrid
3                Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid
4                Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid
                                                ...                                          
3145                   Piso de cuatro habitaciones Calle fernandez de la hoz, Almagro, Madrid
3146                            Piso de dos habitaciones Calle de Sandoval, Trafalgar, Madrid
3147                                                       Estudio nuevo, Prosperidad, Madrid
3148                                          Piso de tres habitaciones 80 m², Aluche, Madrid
3149                                   Piso de cuatro habita

In [44]:
dfiprecios.columns

Index(['url', 'precio'], dtype='object')

In [45]:
#quito precio de dfi
dfi_1=dfi_1.drop(columns="precio")

In [46]:
dfi_1.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'title2', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/

In [47]:
dfi_1.drop_duplicates(inplace=True)

In [48]:
dfiprecios.drop_duplicates(inplace=True)

In [49]:
dfiprecios.columns

Index(['url', 'precio'], dtype='object')

In [50]:
dfi=pd.merge(dfi_1,dfiprecios,on='url')

In [51]:
dfi_1["url"].value_counts(dropna=False)

url
https://www.indomio.es/anuncios/96405397/    4
https://www.indomio.es/anuncios/96088167/    4
https://www.indomio.es/anuncios/96657267/    4
https://www.indomio.es/anuncios/95940193/    2
https://www.indomio.es/anuncios/96510397/    2
                                            ..
https://www.indomio.es/anuncios/96254013/    1
https://www.indomio.es/anuncios/96254707/    1
https://www.indomio.es/anuncios/96256553/    1
https://www.indomio.es/anuncios/96257763/    1
https://www.indomio.es/anuncios/96928501/    1
Name: count, Length: 3057, dtype: int64

In [52]:

dfi_1 = dfi_1.drop_duplicates(subset=['url'], keep='first')
dfiprecios = dfiprecios.drop_duplicates(subset=['url'], keep='first')


In [53]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'title2', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/

In [54]:
#dfi.to_csv("dUnionSilvia02032025.csv",sep=";", index=False)

In [55]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [56]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}

In [57]:
dfpisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4728 entries, 0 to 4727
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 4728 non-null   object 
 1   ubicacion             4728 non-null   object 
 2   P                     4728 non-null   float64
 3   precio                4728 non-null   int64  
 4   habitaciones          4544 non-null   float64
 5   banos                 4469 non-null   float64
 6   m2_constr             4726 non-null   float64
 7   planta                2927 non-null   object 
 8   EUR/m2                4716 non-null   float64
 9   estado                2928 non-null   object 
 10  url                   4728 non-null   object 
 11  fecha_descarga        4728 non-null   object 
 12  tipologia             4728 non-null   object 
 13  web                   4728 non-null   object 
 14  barrio                4709 non-null   object 
 15  cod_barrio           

In [58]:
# Función para extraer desde la palabra clave hasta el final
def extraer_callesincomas(titulo):
    palabras_clave = ["CALLE", "AVENIDA", "PLAZA", "PASEO"]
    titulo = titulo.upper().strip()  # Convertir a mayúsculas y eliminar espacios en blanco al principio y al final
    for palabra in palabras_clave:
        if palabra in titulo:
            return titulo.split(palabra, 1)[1]
    



In [59]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [60]:
dfbarrios["barrio"] = dfbarrios["barrio"].astype(object)
dfbarrios["distrito"] = dfbarrios["distrito"].astype(object)
dfbarrios["cod_barrio"] = dfbarrios["cod_barrio"].astype(int)
dfbarrios["cod_distrito"] = dfbarrios["cod_distrito"].astype(int)
dfbarrios["codigo_Postal"] = dfbarrios["codigo_Postal"].astype(object)
dfbarrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'distrito']]
dfbarrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'barrio']]

In [61]:
dfpisos.columns


Index(['title', 'ubicacion', 'P', 'precio', 'habitaciones', 'banos',
       'm2_constr', 'planta', 'EUR/m2', 'estado', 'url', 'fecha_descarga',
       'tipologia', 'web', 'barrio', 'cod_barrio', 'distrito', 'cod_distrito',
       'codigo_Postal', 'calle', 'espacios', 'detalle', 'actualizacion',
       'referencia', 'disponible_ce', 'letra_ce', 'orientacion', 'jardin',
       'aire_acondicionado', 'calefaccion', 'cocina', 'tipo_suelo', 'piscina',
       'armarios', 'garaje', 'm2_utiles', 'antiguedad', 'gastos_comunidad',
       'puerta_blindada', 'alarma', 'acceso_discapacitado', 'terraza',
       'trastero', 'ascensor', 'balcon', 'exterior', 'amueblado',
       'seguridad 24 h', 'vidrios dobles'],
      dtype='object')

In [62]:
dfpisos["gastos_comunidad"].unique()

array([nan, '250', '290', '65', 'Entre 60 y 80 €', 'Entre 80 y 100 €',
       'Entre 40 y 60 €', 'Entre 20 y 40 €', 'Entre 10 y 20 €',
       'Más de 100 €', '165€ limpieza, pisicina, gimnasio.....', '1.000',
       'No tiene', '37', '205 incluido ibi que esta prorrateado cada mes',
       '130 €', 'Comunidad: 320€ mensual  / ibi 1.040,96€ anual',
       '650 / ibi 2850', '157 / ibi 924', '113,6',
       '125€ (116,37€-134,66€)', 'Trimestral',
       'Gastos comunidad: 205€- ibi: 745€', '103€', '89€ / ibi: 792€',
       '230€  ibi 2.300€', '99€', '100€/mes - ibi 1.050€',
       'Gastos comunidad 385€ ibi 1.500€',
       'Gastos comunidad: 170€ ibi: 1.400€', '50',
       '100€ gastos comunidad/ibi 1.080€', 'G.c: 440€ - ibi: 3.530€',
       '50€ aprox.', '110', '282,16€', '215 +67 para incidencias', '170',
       '25', '98€', '31€/mes', '189', '200€', '230', '300', '136', '99',
       '95,58€', '400€', '57', '78', '78€', '60 €.-', '146,94', '220',
       '149', '95€', '38', 'Calefaccion 

In [63]:
dfpisos.shape

(4728, 49)

In [64]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'title2', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/

In [65]:
dfi.query("derecho_superficie.notnull()")[["url","derecho_superficie"]]

,url,derecho_superficie
1281,https://www.indomio.es/anuncios/96405397/,1.00
1282,https://www.indomio.es/anuncios/96405397/,1.00
1283,https://www.indomio.es/anuncios/96405397/,1.00
1284,https://www.indomio.es/anuncios/96405397/,1.00


In [66]:

# Eliminar filas donde la columna 'derecho_superficie' sea igual a 1.00, por que no es un piso en venta y despues eliminar la columna
dfi = dfi[dfi['derecho_superficie'] != 1.00]
dfi=dfi.drop(columns='derecho_superficie')

In [67]:
dfi=dfi.drop(columns="title2")

In [68]:
dfpisos["armarios"].value_counts(dropna=False)

armarios
NaN                      3588
1                         810
Más de 2                  254
2                          71
1 closet                    1
Mas de 5                    1
4 empotrados                1
8 armarios empotrados       1
3 empotrados                1
Name: count, dtype: int64

In [69]:
#creo la columna tiene armario
dfpisos['tiene_armario'] = dfpisos['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)

In [70]:
dfpisos["tiene_armario"].value_counts(dropna=False)

tiene_armario
NaN     3588
1.00    1140
Name: count, dtype: int64

In [71]:
dfbarrios.head()

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n
0,1,Centro,11,1,Palacio,28013,centro,palacio
1,1,Centro,12,2,Embajadores,28012,centro,embajadores
2,1,Centro,13,3,Cortes,28014,centro,cortes
3,1,Centro,14,4,Justicia,28004,centro,justicia
4,1,Centro,15,5,Universidad,28015,centro,universidad




# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = normalizar_texto(fila['ubicacion'].split('(')[0].strip().replace('/', '')).strip()
    print(bar)
    if bar=='salvador':
        bar='elsalvador'
    if bar=='san andres':
        bar='villaverdealto,cascohistoricodevillaverde'
    if bar=='palos de moguer':
        bar='palosdemoguer'
    if bar=='virgen del cortijo-manoteras':
        bar='valdefuentes'  
    if bar=='sanchinarro':
        bar= 'valdefuentes'    
    if bar=='montecarmelo':
        bar='mirasierra'
    if bar=='ambroz':
        bar='cascohistoricodevicalvaro'
    #para los berrocales como es un barrio nuevo, lo creo aunque o exista oficialmente
    if bar=='los berrocales':
        dfpisos.at[idx, 'cod_barrio'] = 195
       
    valores = obtener_valores_por_columna(dfbarrios, 'barrio_n', bar)
    if valores:
      
        if pd.isna(dfpisos.at[idx, 'barrio']): 
            dfpisos.at[idx, 'barrio'] = valores.get('barrio')       
        if pd.isna(dfpisos.at[idx, 'cod_barrio']): 
            dfpisos.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(dfpisos.at[idx, 'cod_distrito']): 
            dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(dfpisos.at[idx, 'distrito']): 
            dfpisos.at[idx, 'distrito'] = valores.get('distrito')
        if pd.isna(dfpisos.at[idx, 'codigo_Postal']): 
            dfpisos.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')
        dfpisos.at[idx, 'zona'] = 'bar'

In [72]:
def asignar_zona_distrito_barrio(row):
    if row['zona'] == 'Sanchinarro':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'San Andrés':
        row['distrito'] = 'Villaverde'
        row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
    if row['zona'] == 'Virgen del Cortijo':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Montecarmelo':
        row['distrito'] = 'Fuencarral-El Pardo'
        row['barrio'] = 'Mirasierra'

    return row

In [73]:
def procesar_codpostal_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['codigo_Postal'].isna() & df['cod_barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        #valor=df["codigo_Postal"]
        print(fila['cod_barrio'])
        valores = obtener_valores_por_columna(dfbarrios, 'cod_barrio', fila['cod_barrio']) 
        if valores:    
            print(valores.get('codigo_Postal'))        
            df.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')


In [74]:
def procesar_distritos_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['distrito'].isna() & df['barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        valores = obtener_valores_por_columna(dfbarrios, 'barrio', fila['barrio']) 
        if valores:
             df.at[idx, 'cod_barrio'] = valores.get('cod_barrio') 
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('distrito')
             df.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')

             #teniendo codigo de barrio, pongo el resto de columnas sino tienen valor








# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()  | dfpisos['cod_distrito'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = (fila['cod_barrio'])
    print(bar)
    fila = dfbarrios.loc[dfbarrios['cod_barrio'] == bar]
    if not fila.empty:
       valores=fila.iloc[0].to_dict()
       print(valores)
       dfpisos.at[idx, 'barrio'] = valores.get('barrio')
       dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
       dfpisos.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')
       dfpisos.at[idx, 'distrito'] = valores.get('distrito')

In [75]:
# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()

In [76]:
#completo distritos nulos con barrio informado
#procesar_distritos_nulos(dfpisos, dfbarrios)


In [77]:
dfpisos["distrito"].value_counts(dropna=False)

distrito
Centro                 867
Salamanca              762
Chamartín              331
Chamberí               300
Tetuán                 258
Carabanchel            235
Retiro                 209
Puente de Vallecas     202
Fuencarral-El Pardo    196
Ciudad Lineal          190
Moncloa-Aravaca        170
San Blas-Canillejas    167
Latina                 158
Hortaleza              124
Arganzuela             115
Villaverde              98
Villa de Vallecas       90
Vicálvaro               85
Barajas                 66
Usera                   64
Moratalaz               41
Name: count, dtype: int64

In [78]:
dfbarrios["distrito"] = dfbarrios["distrito"].apply(lambda x: x.strip())


In [79]:
dfi.shape,dfpisos.shape

((3146, 74), (4728, 50))

In [80]:
dfpisos.columns

Index(['title', 'ubicacion', 'P', 'precio', 'habitaciones', 'banos',
       'm2_constr', 'planta', 'EUR/m2', 'estado', 'url', 'fecha_descarga',
       'tipologia', 'web', 'barrio', 'cod_barrio', 'distrito', 'cod_distrito',
       'codigo_Postal', 'calle', 'espacios', 'detalle', 'actualizacion',
       'referencia', 'disponible_ce', 'letra_ce', 'orientacion', 'jardin',
       'aire_acondicionado', 'calefaccion', 'cocina', 'tipo_suelo', 'piscina',
       'armarios', 'garaje', 'm2_utiles', 'antiguedad', 'gastos_comunidad',
       'puerta_blindada', 'alarma', 'acceso_discapacitado', 'terraza',
       'trastero', 'ascensor', 'balcon', 'exterior', 'amueblado',
       'seguridad 24 h', 'vidrios dobles', 'tiene_armario'],
      dtype='object')

In [81]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'sup_comercial', 'tipo_inmueble',
       'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exte

In [82]:
# Eliminar la primera columna 
#dfi = dfi.iloc[:,1:]

In [83]:
dfi["web"]="Indomio.com"
dfi["fecha_descarga"]="2024-11-20"

In [84]:
dfi["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN        1494
1900.00     110
1960.00     109
1965.00      83
1970.00      72
           ... 
1863.00       1
2018.00       1
1903.00       1
1800.00       1
1949.00       1
Name: count, Length: 124, dtype: int64

In [85]:
# Convertir a enteros sin afectar los valores nulos
dfi['ano_construccion'] = pd.to_numeric(dfi['ano_construccion'], errors='coerce').dropna().astype('Int64')

In [86]:

cols_anadir=['actualizacion','armarios','codigo_Postal','m2_utiles','referencia','tipo_suelo']
# Añadir columnas a dfi con valores predeterminados (None o NaN)
for col in cols_anadir:
    dfi[col] = None  # Puedes usar np.nan si prefieres

nuevos_nombres ={'armario_empotrado': 'tiene_armario','read_all':'detalle','letrace':'letra_ce','acceso_minusvalido':'acceso_discapacitado' ,'ano_construccion':'antiguedad'} 
dfi = dfi.rename(columns=nuevos_nombres)
dfi['certificado_energetico'] = dfi['letra_ce'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')

In [87]:
nuevos_nombres ={'consumo_energia': 'certificado_energetico','seguridad 24 h':'porteria','vidrios dobles':'carpinteria_exterior_doble_vidrio/pvc','habitaciones':'dormitorios','disponible_ce':'certificado_energetico'
 } 
dfpisos = dfpisos.rename(columns=nuevos_nombres)


In [88]:
#https://www.pisos.com/comprar/apartamento-jeronimos28014-45922451620_101800/ para reformar
#https://www.pisos.com/comprar/apartamento-jeronimos28014-46697950073_101800/ reformado

In [89]:
dfpisos["dormitorios"].value_counts(dropna=False)

dormitorios
3.00     1581
2.00     1379
4.00      645
1.00      520
5.00      236
NaN       184
6.00       90
7.00       52
8.00       21
9.00        8
11.00       4
10.00       3
12.00       3
15.00       2
Name: count, dtype: int64

In [90]:
dfpisos.query("precio.isnull()")

,title,ubicacion,P,precio,dormitorios,banos,m2_constr,planta,EUR/m2,estado,url,fecha_descarga,tipologia,web,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,calle,espacios,detalle,actualizacion,referencia,certificado_energetico,letra_ce,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,m2_utiles,antiguedad,gastos_comunidad,puerta_blindada,alarma,acceso_discapacitado,terraza,trastero,ascensor,balcon,exterior,amueblado,porteria,carpinteria_exterior_doble_vidrio/pvc,tiene_armario


In [91]:
dfpisos.shape,dfi.shape

((4728, 50), (3146, 83))

In [92]:
# Reemplazar "Sí" por 1 y "No" por 0 
dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})
dfpisos["zona"]=dfpisos["ubicacion"]

# Actualizar 'nuda_propiedad' con el valor de 'usufructo' cuando 'usufructo' es 1 y 'nuda_propiedad' es NaN
dfi.loc[(dfi['usufructo'] == 1) & (dfi['nuda_propiedad'].isna()), 'nuda_propiedad'] = dfi['usufructo']

# Definir las columnas a eliminar
columnas_a_eliminar = ['habitaciones','usufructo','buhardilla','hidromasaje', 'ciudad' ,'instalacion_tv_individual','instalacion_tv_centralizada']

# Eliminar las columnas
dfi = dfi.drop(columns=columnas_a_eliminar)


# Definir las columnas a eliminar
columnas_a_eliminar = ['ubicacion']

# Eliminar las columnas
dfpisos = dfpisos.drop(columns=columnas_a_eliminar)



C:\Users\extas\AppData\Local\Temp\ipykernel_2224\3244788326.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
C:\Users\extas\AppData\Local\Temp\ipykernel_2224\3244788326.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})


In [93]:
dfpisos.columns

Index(['title', 'P', 'precio', 'dormitorios', 'banos', 'm2_constr', 'planta',
       'EUR/m2', 'estado', 'url', 'fecha_descarga', 'tipologia', 'web',
       'barrio', 'cod_barrio', 'distrito', 'cod_distrito', 'codigo_Postal',
       'calle', 'espacios', 'detalle', 'actualizacion', 'referencia',
       'certificado_energetico', 'letra_ce', 'orientacion', 'jardin',
       'aire_acondicionado', 'calefaccion', 'cocina', 'tipo_suelo', 'piscina',
       'armarios', 'garaje', 'm2_utiles', 'antiguedad', 'gastos_comunidad',
       'puerta_blindada', 'alarma', 'acceso_discapacitado', 'terraza',
       'trastero', 'ascensor', 'balcon', 'exterior', 'amueblado', 'porteria',
       'carpinteria_exterior_doble_vidrio/pvc', 'tiene_armario', 'zona'],
      dtype='object')

In [94]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpi

que tengo en dfi que no tengo en dfpisos

['precio_anterior', 'descuento', 'zona', 'calle', 'consumoce_ano', 
 , 'emisiones_co2', 'propiedad_completa', 
 'dormitorios', 'porteria', 'plantas_edificio', 'inmueble_ingresos', 
 'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo', 'nuda_propiedad',
 , 'sup_comercial', 'buhardilla', 
 'cancha_tenis', 'carpinteria_exterior_doble_vidrio/madera', 
 'carpinteria_exterior_doble_vidrio/metal', 
 'carpinteria_exterior_triple_vidrio/madera', 
 'carpinteria_exterior_triple_vidrio/metal', 
 'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera', 
 'carpinteria_exterior_vidrio/metal', 'chimenea', 'fibra_optica',  'hidromasaje',  
 'interior_y_exterior', 'porton_electrico', 'puerta_blindada', 'alarma', 
 'videoportero', 'lujosa', ]

 que tengo en dfpisos que no tengo en dfi

 [ 'referencia', 'fecha_publicacion',  'm2_utiles', 'tipo_suelo' ]



In [95]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3146 entries, 0 to 3149
Data columns (total 76 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 3146 non-null   int64  
 1   title                                      3146 non-null   object 
 2   url                                        3146 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  55 non-null     object 
 5   EUR/m2                                     3140 non-null   float64
 6   m2_constr                                  3140 non-null   float64
 7   distrito                                   3044 non-null   object 
 8   cod_distrito                               3146 non-null   int64  
 9   barrio                                     3044 non-null   object 
 10  cod_barrio                   

In [96]:
dfi["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
No indicado    1835
Disponible     1311
Name: count, dtype: int64

In [97]:
dfpisos.shape,dfi.shape

((4728, 50), (3146, 76))

In [98]:
# Obtén las columnas exclusivas de dfpisos
columnas_solo_en_dfpisos = dfpisos.columns.difference(dfi.columns)
print(columnas_solo_en_dfpisos)


Index(['P', 'espacios', 'gastos_comunidad'], dtype='object')


In [99]:
# Asegurarse de que ambos dataframes tengan las mismas columnas, rellenando con NaN 
dfpisos = dfpisos.reindex(columns=dfi.columns, fill_value=None) 
dfi = dfi.reindex(columns=dfpisos.columns, fill_value=None) 

In [100]:
dfpisos.shape,dfi.shape

((4728, 76), (3146, 76))

In [101]:
dfpisos.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpi

In [102]:
dfpisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4728 entries, 0 to 4727
Data columns (total 76 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 0 non-null      float64
 1   title                                      4728 non-null   object 
 2   url                                        4728 non-null   object 
 3   precio_anterior                            0 non-null      float64
 4   descuento                                  0 non-null      float64
 5   EUR/m2                                     4716 non-null   float64
 6   m2_constr                                  4726 non-null   float64
 7   distrito                                   4728 non-null   object 
 8   cod_distrito                               4728 non-null   int64  
 9   barrio                                     4709 non-null   object 
 10  cod_barrio              


# # Sumar las filas 

#dfi['precio'] = dfi['precio'].str.replace('.', '').str.replace(',', '')
numeric_columns = ['cod_distrito','cod_barrio','trastero','dormitorios','EUR/m2'] # Añade aquí todas las columnas numéricas 
dfi[numeric_columns] = dfi[numeric_columns].astype(float) 
dfpisos[numeric_columns] = dfpisos[numeric_columns].astype(float)

non_numeric_columns = ['tipologia','detalle','precio','banos','piscina','garaje','planta','porteria','amueblado','ano_construccion','title', 'url', 'distrito', 'estado','descuento','letrace','emisiones_co2','consumoce_ano','tipo_inmueble','zona','calle'] # Añade aquí todas las columnas no numéricas
dfi[non_numeric_columns] = dfi[non_numeric_columns].astype(str) 
dfpisos[non_numeric_columns] = dfpisos[non_numeric_columns].astype(str)

# # Sumar las filas 
df_union = pd.concat([dfi, dfpisos], ignore_index=True)
#df_union = dfi.add(dfpisos, fill_value=None)

In [103]:
# # Sumar las filas 
df_union = pd.concat([dfi, dfpisos], ignore_index=True)

C:\Users\extas\AppData\Local\Temp\ipykernel_2224\3932775271.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_union = pd.concat([dfi, dfpisos], ignore_index=True)


In [104]:
df_union.shape

(7874, 76)

In [105]:
df_union.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpi

In [106]:
df_union["piscina"].value_counts(dropna=False)

piscina
NaN                                          4511
0                                            2749
1                                             397
Comunitaria                                   139
Propia                                         76
1 piscina de adultos y 1 piscina de niños       1
Con salorium                                    1
Name: count, dtype: int64

In [107]:

# Reemplazar los valores según las indicaciones
df_union['piscina'] = df_union['piscina'].replace({
    '-nan': 'NaN',
    'NaN': 'NaN',
    '-Comunitaria': 'Comunitaria',
    '-Propia': 'Privada',
    '-Con salorium': 'Comunitaria',
    '-1 piscina de adultos y 1 piscina de niños': 'Comunitaria'
})



In [108]:
df_union["jardin"].value_counts(dropna=False)

jardin
NaN                             6612
Sin jardín                       859
Jardín privado                   150
Comunitario                      114
Privado                           91
Jardín comunitario                44
Jardín privado y comunitario       4
Name: count, dtype: int64

In [109]:
df_union['tiene_jardin'] = df_union['jardin'].apply(lambda x: 1 if pd.notnull(x) and x != 'Sin jardín' else (0 if x == 'Sin jardín' else None))

In [110]:
df_union['jardin'] = df_union['jardin'].replace({
    'Jardín privado': 'Privado',
    'Jardín comunitario': 'Comunitario',
    'Jardín privado y comunitario': 'Privado'
})


In [111]:
df_union["tipologia"].value_counts(dropna=False)

tipologia
Piso                    6650
Ático                    330
Chalet                   278
Dúplex                   149
Apartamento              129
Casa unifamiliar         102
Casa                      89
Estudio                   73
Casa adosada              42
Loft                      18
Casa pareada               8
Casa rústica               3
Casa plurifamiliar         1
Caserío                    1
Apartamento en villa       1
Name: count, dtype: int64

In [112]:
#Reemplazar todas las tipologías que comienzan con "Casa" por "Casa" 
df_union['tipologia'] = df_union['tipologia'].replace(r'^Casa.*', 'Casa', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Apartamento.*', 'Apartamento', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Caserío.*', 'Casa', regex=True)

In [113]:
df_union["precio"].value_counts(dropna=False)

precio
650000     58
950000     53
1200000    52
2500000    51
829000     47
           ..
1374900     1
1399900     1
1704900     1
998900      1
211000      1
Name: count, Length: 1395, dtype: int64

In [114]:

# Eliminar las filas donde 'precio' es nulo 
df_union = df_union.dropna(subset=['precio'])

In [115]:
df_union.columns

Index(['Unnamed: 0', 'title', 'url', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'antiguedad', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpi

In [116]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7874 entries, 0 to 7873
Data columns (total 77 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 3146 non-null   float64
 1   title                                      7874 non-null   object 
 2   url                                        7874 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  55 non-null     object 
 5   EUR/m2                                     7856 non-null   float64
 6   m2_constr                                  7866 non-null   float64
 7   distrito                                   7772 non-null   object 
 8   cod_distrito                               7874 non-null   int64  
 9   barrio                                     7753 non-null   object 
 10  cod_barrio              

In [117]:
df_union["descripcion_larga"]=df_union["detalle"]
df_union=df_union.drop(columns="detalle")

In [118]:
# Aplicar la función solo si 'calle' es nulo
df_union['calle'] = df_union.apply(
    lambda row: extraer_callesincomas(row['title']) if pd.isnull(row['calle']) else row['calle'],
    axis=1
)


In [119]:
df_union.query("barrio.isnull()")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
11,11.00,"Piso de tres habitaciones buen estado, planta baja, San Andrés, Madrid",https://www.indomio.es/anuncios/83875795/,NaN,NaN,2.20,75.00,NaN,0,NaN,0.00,San Andrés,None,"≥ 3,51",F,Pendiente,Piso,1.00,Bueno / Habitable,Sí,1,3.00,1,1.00,1.00,NaN,1.00,0.00,NaN,"Individual, por radiadores, de gas",1971,4.00,"Individual, frío",Sin jardín,NaN,1.00,Libre,NaN,NaN,medio,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,Cocina independiente,1.00,1.00,0.00,0.00,0,1.00,0,0,0.00,0.00,55,1.00,NaN,165000,Indomio.com,2024-11-20,None,None,NaN,None,None,None,Disponible,0.00,"Piso en venta en edificación de buen estado pasada la ITE, planta baja.cuenta con 3 habitaciones amplias,un salón de 15m2un comedor independiente de 10m2 con vista exterioruna cocina amplia necesidad de modernizar,un amplio bañouna terraza de 20m2 que es de uso exclusivoun trastero tamaño para hacer otra habitación independiente.el piso en total de uso útil es de 90m2.potencial para crecer y hacer otros espacios.Todo esta saneado sin ninguna clase de cargas."
53,53.00,"Piso de dos habitaciones buen estado, Sanchinarro, Madrid",https://www.indomio.es/anuncios/89878173/,NaN,NaN,5.38,79.00,NaN,0,NaN,0.00,Sanchinarro,None,desconocido,D,No indicado,Piso,NaN,Bueno / Habitable,Sí,1,2.00,2,NaN,NaN,1 en aparcamiento/garaje público,1.00,1.00,NaN,Individual,2005,NaN,"Individual, frío/calor",Privado,NaN,NaN,NaN,NaN,162.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,1.00,NaN,425000,Indomio.com,2024-11-20,None,None,NaN,None,None,None,Disponible,1.00,"Se vende piso de particular a particular. Se vende piso en Sanchinarro. Estupendo inmueble en la quinta planta de un edificio de seis alturas, con 83 metros construidos, con dos habitaciones con armarios empotrados, dos cuartos de baño, uno con ducha y otro completo con bañera. Cocina con terraza cerrada totalmente amueblada. Salón con mirador. También dispone de dos plazas de garaje amplias, para moto + coche cada una, y un amplio trastero. La urbanización consta de gran piscina, zonas ajardinadas, gimnasio, sauna y vigilancia las 24 horas. El inmueble está situado próximo a dos hospitales y tiene todos los servicios de bancos y supermercados en el barrio, además de colegios y centro cultural. Buena comunicación con otras zonas a través de autobús y metro-ligero. Se vende con una hipoteca ya contratada y en la que se subrogaría el comprador. También es un piso ideal para inversores. El régimen de visitas, se hará a través de una lista de contactos por orden de llamada a partir del 15 de noviembre. Abstenerse AGENCIAS. Gracias."
70,70.00,"Piso de tres habitaciones Calle De Dulce Chacón, Virgen del Cortijo-Manoteras, Madrid",https://www.indomio.es/anuncios/90956999/,NaN,NaN,4.41,204.00,NaN,0,NaN,0.00,Virgen del Cortijo,calle de dulce chacon,desconocido,E,E,Piso,NaN,Bueno / Habitable,NaN,1


# Para 'Piso en venta en Cármenes'
tit = 'Piso en venta en Cármenes'
df_union.loc[
    (df_union["title"] == tit) & (df_union["cod_barrio"].isnull()),
    ['cod_barrio', 'barrio', 'cod_distrito', 'distrito', 'codigo_Postal']
] = [101, 'Los Cármenes', 10, 'Latina', 28047]

# Para 'Apartamento en venta en Calle de los Chisperos'
tit = 'Apartamento en venta en Calle de los Chisperos'
df_union.loc[
    (df_union["title"] == tit) & (df_union["cod_barrio"].isnull()),
    ['cod_barrio', 'barrio', 'cod_distrito', 'distrito', 'codigo_Postal']
] = [101, 'Los Cármenes', 10, 'Latina', 28047]


In [120]:
dfcalles= pd.read_excel('../02-datos-limpios/CALLEJERO_VIGENTE_NUMERACIONES_202501.xlsx')

In [121]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [122]:
#nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
#df_pisos[nuevas_columnas]=np.nan



# Cambiar el tipo de dato de las columnas "barrio" y "distrito" a object
df_union["barrio"] = df_union["barrio"].astype(object)
df_union["distrito"] = df_union["distrito"].astype(object)
df_union["cod_barrio"] = df_union["cod_barrio"].astype(object)
df_union["cod_distrito"] = df_union["cod_distrito"].astype(object)
df_union["codigo_Postal"] = df_union["codigo_Postal"].astype(object)
dfbarrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'distrito']]
dfbarrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'barrio']]

def clasificar_distrito_y_barrio_por_columnas(df, df_barrios):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Recorrer las columnas en orden de prioridad: titulo, nombre_oficina
        for columna in ['title']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)

                 # Reemplazar valores específicos en la ubicación normalizada
                if 'sanandres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanandres', 'villaverdealto')
                elif 'salvador' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('salvador', 'elsalvador')
                elif 'san andres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('san andres', 'villaverdealto')
                elif 'palos de moguer' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('palos de moguer', 'palosdemoguer')
                elif 'virgendelcortijo-manoteras' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('virgendelcortijo-manoteras', 'valdefuentes')
                elif 'sanchinarro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanchinarro', 'valdefuentes')
                elif 'montecarmelo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo', 'mirasierra')
                elif 'ambroz' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('ambroz', 'cascohistoricodevicalvaro')
                elif 'carmenes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('carmenes', 'loscarmenes')
                elif 'paudecarabanchel' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('paudecarabanchel', 'buenavista')
                #elif 'centrohistorico' in valor_normalizado:
                    #valor_normalizado = valor_normalizado.replace('centrohistorico', 'cascohistoricodevicalvaro')
                elif 'lastablas' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lastablas', 'valverde')
                
                # Verificamos si el valor normalizado contiene el nombre de un barrio
                barrio_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal
                # Verificamos si el valor normalizado contiene el nombre de un distrito
                distrito_encontrado = next((distrito for distrito in df_barrios["distrito_n"] if distrito in valor_normalizado), None)
                if distrito_encontrado and pd.isna(fila["distrito"]):
                    df.at[idx, "distrito"] = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")["distrito"].values[0]
                    result = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")                    
                    cod_distrito = result["cod_distrito"].values[0]
                    df.at[idx, "cod_distrito"] = cod_distrito
            else:
                #zona_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                barrio_encontrado = next((zona_barrio_dict[zona]["barrio"] for zona in zona_barrio_dict if zona_barrio_dict[zona]["barrio"] in df_barrios["barrio_n"]), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal

In [123]:
clasificar_distrito_y_barrio_por_columnas(df_union, dfbarrios)

In [124]:
df_union.query("barrio.isnull()")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
3898,NaN,"Piso en venta en Avenida Levantge, S/N",https://www.pisos.com/comprar/piso-los_berrocales28052-4592114246_109700/,NaN,NaN,3218.00,110.00,Vicálvaro,19,NaN,NaN,Los Berrocales (Distrito Vicálvaro. Madrid Capital),"LEVANTGE, S/N",NaN,NaN,NaN,Piso,NaN,A estrenar,NaN,1,3.00,2.00,NaN,1.00,NaN,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354000,pisos.com,19/10/2024,21/10/2024,NaN,NaN,NaN,Ref.: 4990150-MHI1,NaN,0.00,NaN,"VI-1-2-1-P01-A. Piso en un edificio de obra nueva de 110 m2 de superficie construida y ubicado en la zona de Madrid Capital, enclavado en planta 1. Está distribuido en 3 habitaciones, 2 baños completos y terraza de 7,3 m2. Incluye extras como ascensor, garaje y trastero.** El precio mostrado en la tabla anterior para cada inmueble, incluye únicamente aquellos anejos (plazas de garaje y/o trastero) que hubiesen sido expresamente especificados en su descripción. En caso de no aparecer especificados, su precio no se entenderá incluido en el precio mostrado anteriormente. El precio mostrado lo es sin incluir impuestos. Los inmuebles identificados están sujetos, en todo caso, a disponibilidad."
4064,NaN,"Piso en venta en Calle Berrocales Ix, 461",https://www.pisos.com/comprar/piso-los_berrocales28031-4596004879_109700/,NaN,NaN,3862.00,77.00,Vicálvaro,19,NaN,NaN,Los Berrocales (Distrito Vicálvaro. Madrid Capital),"BERROCALES IX, 461",NaN,NaN,NaN,Piso,NaN,A estrenar,NaN,NaN,2.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,297400,pisos.com,19/08/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4065,NaN,"Piso en venta en Calle Berrocales Ix, 461",https://www.pisos.com/comprar/piso-los_berrocales28031-4849665223_109700/,NaN,NaN,3915.00,77.00,Vicálvaro,19,NaN,NaN,Los Berrocales (Distrito Vicálvaro. Madrid Capital),"BERROCALES IX, 461",NaN,NaN,NaN,Piso,NaN,A estrenar,NaN,NaN,2.00,2.00,NaN,1.00,NaN,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,301500,pisos.com,19/10/2024,10/04/2024,NaN,NaN,NaN,Ref.: 4993266-0302,NaN,0.00,NaN,"P2_1ºC. Vivienda de 2 dormitorios en Planta Primera, 77m² construidos, 57,50m² útiles y terraza 6,80m². Con precio desde 301.500€ más (10% IVA), 1 plaza de garaje y 1 trastero incluido en el precio."
4066,NaN,"Piso en venta en Calle Berrocales Ix, 461",https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,NaN,NaN,3736.00,99.00,Vicálvaro,19,NaN,NaN,Los Berrocales (Distrito Vicálvaro. Madrid Capital),"BERROCALES IX, 461",NaN,NaN,NaN,Piso,NaN,A estrenar,NaN,NaN,3.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [125]:
df_union.query("cod_barrio.notnull()")[["web","title","cod_barrio","cod_distrito","barrio","distrito","codigo_Postal"]]

,web,title,cod_barrio,cod_distrito,barrio,distrito,codigo_Postal
0,Indomio.com,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",41.00,4,Recoletos,Salamanca,NaN
1,Indomio.com,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",213.00,21,Casco Histórico de Barajas,Barajas,NaN
2,Indomio.com,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",24.00,2,Legazpi,Arganzuela,NaN
3,Indomio.com,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",54.00,5,Hispanoamérica,Chamartín,NaN
4,Indomio.com,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",54.00,5,Hispanoamérica,Chamartín,NaN
...,...,...,...,...,...,...,...
7868,pisos.com,Ático en venta en Valdemarín,95.00,9,Valdemarín,Moncloa-Aravaca,28023.00
7869,pisos.com,Ático en venta en Vallecas,181.00,18,Casco Histórico de Vallecas,Villa de Vallecas,28031.00
7870,pisos.com,Ático en venta en Vallehermoso,61.00,6,Bellas Vistas,Tetuán,28039.00
7871,pisos.com,Ático en venta en Ventas,151.00,15,Ventas,Ciudad Lineal,28027.00


In [126]:
df_union.query("codigo_Postal.isna()").count()

Unnamed: 0                3044
title                     3063
url                       3063
precio_anterior             53
descuento                   53
                          ... 
referencia                  13
tipo_suelo                   0
certificado_energetico    3057
tiene_jardin              1017
descripcion_larga         3015
Length: 77, dtype: int64

In [127]:
df_union["cod_barrio"].value_counts(dropna=False)

cod_barrio
42.00     383
15.00     359
46.00     321
41.00     320
11.00     247
         ... 
27.00       5
212.00      4
193.00      3
192.00      2
142.00      1
Name: count, Length: 131, dtype: int64

In [128]:
df_union.loc[df_union['cod_barrio'].notnull(), 'cod_barrio'] = df_union.loc[
    df_union['cod_barrio'].notnull(), 'cod_barrio'
].astype(int)

In [129]:
dfbarrios["cod_barrio"].value_counts(dropna=False)

cod_barrio
11     1
135    1
157    1
156    1
155    1
      ..
72     1
71     1
66     1
65     1
215    1
Name: count, Length: 132, dtype: int64

In [130]:
#voy a asignar codigo postal teniendo cod_barrio
procesar_codpostal_nulos(df_union, dfbarrios)


41
28001
213
28042
24
28045
54
28016
54
28016
54
28016
54
28016
62
28020
51
28002
42
28001
72
28015
105
28024
105
28024
46
28006
14
28004
87
28035
54
28016
183
28031
132
28053
72
28015
41
28001
16
28013
74
28010
45
28006
51
28002
54
28016
54
28016
55
28016
45
28006
41
28001
41
28001
15
28015
42
28001
55
28016
73
28010
15
28015
43
28028
56
28046
56
28046
54
28016
35
28014
55
28016
16
28013
16
28013
41
28001
41
28001
41
28001
63
28020
51
28002
206
28037
11
28013
153
28027
55
28016
46
28006
55
28016
12
28012
74
28010
92
28008
62
28020
41
28001
41
28001
91
28011
52
28002
112
28019
15
28015
16
28013
41
28001
51
28002
75
28003
12
28012
74
28010
74
28010
41
28001
42
28001
15
28015
15
28015
207
28037
34
28009
32
28007
66
28039
164
28033
13
28014
62
28020
16
28013
51
28002
181
28031
41
28001
194
28032
194
28032
183
28031
15
28015
107
28044
127
28041
163
28043
44
28028
181
28031
42
28001
44
28028
15
28015
164
28033
72
28015
121
28041
12
28012
45
28006
12
28012
102
28011
152
28017
63
28020
34
280

In [131]:
df_union.query("codigo_Postal.isna()")[["url","title","cod_barrio","cod_distrito","codigo_Postal"]]

,url,title,cod_barrio,cod_distrito,codigo_Postal
3898,https://www.pisos.com/comprar/piso-los_berrocales28052-4592114246_109700/,"Piso en venta en Avenida Levantge, S/N",NaN,19,NaN
4064,https://www.pisos.com/comprar/piso-los_berrocales28031-4596004879_109700/,"Piso en venta en Calle Berrocales Ix, 461",NaN,19,NaN
4065,https://www.pisos.com/comprar/piso-los_berrocales28031-4849665223_109700/,"Piso en venta en Calle Berrocales Ix, 461",NaN,19,NaN
4066,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,"Piso en venta en Calle Berrocales Ix, 461",NaN,19,NaN
4067,https://www.pisos.com/comprar/piso-los_berrocales28031-4891530918_109700/,"Piso en venta en Calle Berrocales Ix, 461",NaN,19,NaN
4196,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/,"Piso en venta en Calle Parcela Rc.4.6.2. Sector Uzpp. 02.04, S/N",NaN,19,NaN
4197,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/,"Piso en venta en Calle Parcela Rc.4.6.2. Sector Uzpp. 02.04, S/N",NaN,19,NaN
7616,https://www.pisos.com/comprar/piso-los_berrocales28031-4677433362_109700/,Piso en venta en los Berrocales,NaN,19,NaN
7617,https://www.pisos.com/comprar/piso-los_berrocales28031-4588865732_109700/,Piso en venta en los Berrocales,NaN,19,NaN
7618,https://www.pisos.com/comprar/piso-los_berrocales28031-4355464537_109700/,Piso en venta en los Berrocales,NaN,19,NaN


In [132]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7874 entries, 0 to 7873
Data columns (total 77 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 3146 non-null   float64
 1   title                                      7874 non-null   object 
 2   url                                        7874 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  55 non-null     object 
 5   EUR/m2                                     7856 non-null   float64
 6   m2_constr                                  7866 non-null   float64
 7   distrito                                   7874 non-null   object 
 8   cod_distrito                               7874 non-null   object 
 9   barrio                                     7855 non-null   object 
 10  cod_barrio              

In [133]:
df_union["precio"].value_counts(dropna=False)

precio
650000     58
950000     53
1200000    52
2500000    51
829000     47
           ..
1374900     1
1399900     1
1704900     1
998900      1
211000      1
Name: count, Length: 1395, dtype: int64

In [134]:
df_union.query("precio==19950000.00")[["web","precio"]]

,web,precio
1704,Indomio.com,19950000
7671,pisos.com,19950000


In [135]:
#df_union['precio'] = df_union['precio'].apply(lambda x: str(x).replace('.', '').strip() if isinstance(x, str) and float(x.replace('.', '')) >= 1000000 else x)

In [136]:

#df_union['precio'] = df_union['precio'].apply(lambda x: str(x).replace('.', '').strip() if isinstance(x, str) and float(x.replace('.', '')) >= 1000000 else x)

# Eliminar puntos de los separadores de miles y convertir a enteros
#df_union['precio'] = df_union['precio'].apply(lambda x: int(str(x).replace('.', '').strip()))




In [137]:
df_union["precio"].describe()

count       7874.00
mean     1021566.74
std      1273804.69
min            1.00
25%       270000.00
50%       629000.00
75%      1275000.00
max     19950000.00
Name: precio, dtype: float64

In [138]:
# elimino la vivienda con precio 1, es lo que aparece en la web

In [139]:
# Filtrar filas donde el precio no es 1
df_union = df_union[df_union['precio'] != 1]

In [140]:
df_union["precio"].describe()

count       7873.00
mean     1021696.49
std      1273833.55
min        25000.00
25%       270000.00
50%       629000.00
75%      1275000.00
max     19950000.00
Name: precio, dtype: float64

In [141]:
df_union.query("precio==25000")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
1226,1226.00,"Piso de tres habitaciones Calle del General Ricardos 46, San Isidro, Madrid",https://www.indomio.es/anuncios/96375609/,NaN,NaN,313.00,80.00,Carabanchel,11,San Isidro,113,San Isidro,calle del general ricardos 46,desconocido,NaN,No indicado,Piso,NaN,Para reformar,Parcialmente amueblado,3,3.00,1,1.00,NaN,NaN,0.00,1.00,NaN,Individual,1960,6.00,NaN,NaN,NaN,NaN,NaN,1.00,104.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,100,NaN,norte,25000,Indomio.com,2024-11-20,None,None,28019,None,None,None,No indicado,NaN,"POR FAVOR , LEER ATENTAMENTE . VENTA 25% NUDA PROPIEDAD , resto del 50% mi madre y otro 25% hermana .25.000 € . Sin cargas . VEndo mi 25% de NUDA PROPIEDAD Usufructo de mi madre , persona 83 años NO visitable Perfecto inversión y constitución societariaContacto preferible Whatsapp"


In [142]:
#elimino esta vivienda por que solo se vende el 25% de la misma
#df_union = df_union[df_union['precio'] != 25000]

In [143]:
df_union.query("precio==59000")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
1570,1574.00,"Casa unifamiliar Calle Rambla, Las Águilas, Madrid",https://www.indomio.es/anuncios/96538071/,NaN,NaN,472.00,125.00,Latina,10,Las Águilas,107,Las Águilas,calle rambla,desconocido,D,No indicado,Casa,NaN,NaN,NaN,NaN,4.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,59000,Indomio.com,2024-11-20,None,None,28044,None,None,None,Disponible,0.00,"Amplia casa en Murcia, perfecto para familias. La vivienda consta de 125 m construidos distribuidos en cuatro habitaciones, salón-comedor, cocina y un aseo. Localizada en el centro de Murcia cerca de Plaza de Las Flores, Real Casino de Murcia y Floridablanca Garden, en sus alrededores encontramos el University Hospital Reina Sofia, la Universidad de Murcia, supermercados, bancos, bibliotecas, farmacias, oficinas de correos, centros deportivos y varias paradas de autobús. Cuenta con fácil acceso a N-340. Oportunidad de inversión: inmueble procedente de ejecución hipotecaria sin posesión. Consulte con nuestro Agente Comercial la información adicional sobre el inmueble y las condiciones especiales de compraventa. El precio del inmueble ha sido fijado en atención a que el inmueble se adquiera por el comprador en estado de ocupado por lo que no aplicaría dicho precio si en el momento de formalización de la escritura pública el inmueble se encontrara libre de ocupantes. Imposibilidad de visita interior. Las fotografías pueden no corresponderse con el estado actual del inmueble."
2369,2373.00,"Piso de una habitación Pico Cejo, Numancia, Madrid",https://www.indomio.es/anuncios/96761591/,NaN,NaN,1.97,30.00,Puente de Vallecas,13,Numancia,136,Numancia,None,desconocido,NaN,No indicado,Piso,NaN,NaN,NaN,NaN,1.00,1,NaN,0.00,NaN,0.00,0.00,NaN,NaN,1910,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,59000,Indomio.com,2024-11-20,None,None,28038,None,None,None,No indicado,0.00,"OPORTUNIDAD VENTA SIN POSESIÓN! El inmueble no se puede visitar interiormente. Este tipo de compra no admite hipoteca. LEA LAS CONDICIONES a continuación. Magnífica oportunidad de piso situado en Madrid, con una inmensa variedad de servicios, tanto de ocio como de necesidad, a su alrededor. La vivienda tiene una superficie construida de 30 m2 y útil 29 m2. Distribuido interiormente en diferentes dependencias y servicios. No dispone de ascensor en el edificio. LA FOTO NO IDENTIFICA LA PROPIEDAD EXACTA DE VENTA, SOLO INFORMA DE LA ZONA. CONDICIONES ESPECIALES DE COMPRA: En este tipo de adquisiciones se requieren una serie de requisitos detallados a continuación: 1º.- No permite hipoteca sobre el inmueble, los bancos no financian estas operaciones. 2º.- VENTA SIN POSESIÓN. LA PROPIEDAD NO SE PUEDE 

In [144]:
df_union["precio"].value_counts(dropna=False)

precio
650000     58
950000     53
1200000    52
2500000    51
829000     47
           ..
1399900     1
1704900     1
998900      1
1064900     1
211000      1
Name: count, Length: 1394, dtype: int64

In [145]:
df_union = df_union[df_union['url'] != 'https://www.indomio.es/anuncios/96761685/']

In [146]:
df_union["tipologia"].value_counts(dropna=False)

tipologia
Piso           6648
Ático           330
Chalet          278
Casa            246
Dúplex          149
Apartamento     130
Estudio          73
Loft             18
Name: count, dtype: int64

In [147]:
df_union.describe()

,Unnamed: 0,precio_anterior,EUR/m2,m2_constr,propiedad_completa,dormitorios,balcon,terraza,trastero,ascensor,plantas_edificio,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,sup_comercial,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,fibra_optica,exterior,interior,interior_y_exterior,porton_electrico,videoportero,lujosa,precio,tiene_jardin
count,3144.00,55.00,7855.00,7865.00,113.00,7645.00,1455.00,2480.00,2285.00,3731.00,211.00,6.00,3.00,3.00,343.00,4284.00,3144.00,3502.00,3144.00,3144.00,3144.00,3144.00,3144.00,3144.00,3144.00,3144.00,3144.00,4449.00,3144.00,3144.00,3144.00,3144.00,3144.00,7872.00,1260.00
mean,1574.87,830694.55,3745.76,145.81,1.00,2.90,0.58,0.77,0.41,0.89,5.51,1.00,1.00,1.00,271.33,0.42,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.32,0.01,0.00,0.01,0.02,0.45,1021819.42,0.32
std,910.05,977555.73,3996.23,138.26,0.00,1.47,0.49,0.42,0.49,0.32,2.60,0.00,0.00,0.00,227.74,0.49,0.03,0.31,0.04,0.06,0.02,0.03,0.05,0.06,0.07,0.12,0.11,0.47,0.08,0.05,0.07,0.15,0.50,1273867.77,0.47
min,0.00,98100.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25000.00,0.00
25%,786.75,201500.00,7.08,71.00,1.00,2.00,0.00,1.00,0.00,1.00,4.00,1.00,1.00,1.00,106.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,270375.00,0.00
50%,1576.50,440000.00,3020.00,105.00,1.00,3.00,1.00,1.00,0.00,1.00,5.00,1.00,1.00,1.00,203.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,629000.00,0.00
75%,2362.25,955000.00,6360.00,165.00,1.00,4.00,1.00,1.00,1.00,1.00,7.00,1.00,1.00,1.00,428.50,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1275000.00,1.00
max,3149.00,4900000.00,27708.00,3250.00,1.00,24.00,1.00,1.00,1.00,1.00,23.00,1.00,1.00,1.00,946.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,19950000.00,1.00


In [148]:
dfisolometros.query("url=='https://www.indomio.es/anuncios/91884849/'")

,url,m2_constr
1409,https://www.indomio.es/anuncios/91884849/,1775.00


In [149]:
df_union.query("banos.isnull()")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
2,2.00,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,NaN,NaN,500.00,400.00,Arganzuela,2,Legazpi,24,Legazpi,plaza de italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,<NA>,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,200000,Indomio.com,2024-11-20,None,None,28045,None,None,None,No indicado,1.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio
72,72.00,"Casa unifamiliar Venta, Embajadores-Lavapiés, Madrid",https://www.indomio.es/anuncios/91050501/,NaN,NaN,NaN,NaN,Centro,1,Embajadores,12,Embajadores,None,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,0.00,NaN,NaN,<NA>,NaN,"Individual, frío/calor",Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,550000,Indomio.com,2024-11-20,None,None,28012,None,None,None,No indicado,0.00,"FINCAS FERNANDEZ pone a la Venta de Edificio con Local bar restaurante y 5 estudios con cocina y oficina amueblados, consta de planta baja el restaurante bar equipado con muebles hostelería y cocina equipada y listo para funcionar, da a dos calles. Planta alta hay oficina con buhardilla y 5 estudios con cocina amueblados con electrodomésticos y esta funcionando, esta dando beneficios, tiene 2 sótanos, en uno esta la caldera de gasoil para agua caliente del edificio y otro para el almacén. El restaurante bar y los estudios son independientes, consta de licencias que están activas, mas informacion por telefono. Los mejores inmuebles y el mejor servicio en FINCAS FERNANDEZ, más de 15 años trabajando en el municipio. Si necesita más información puede pasar a visitarnos a Calle SAIZ ROMILLO 11, o ponerse en contacto a través del 916607413"
119,119.00,"Piso de una habitación buen estado, Embajadores-Lavapiés, Madrid",https://www.indomio.es/anuncios/92627639/,NaN,NaN,4.69,32.00,Centro,1,Embajadores,12,Embajadores,None,desconocido,F,No indicado,Piso,NaN,Bueno / Habitable,No,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,"Individual, frío/calor",Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,150000,Indomio.com,2024-11-20,None,None,28012,None,None,None,Disponible,0.00,"Se vende piso de parti

In [150]:
# Filtrar y actualizar valores
df_union[df_union['m2_constr'] < 13] = actualizar_valores(df_union[df_union['m2_constr'] < 13], dfisolometros, 'url', 'm2_constr')
df_union[df_union['m2_constr'] < 13] = actualizar_valores(df_union[df_union['m2_constr'] < 13], dfpcomlimpio, 'url', 'm2_constr')

C:\Users\extas\AppData\Local\Temp\ipykernel_2224\3684329844.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
C:\Users\extas\AppData\Local\Temp\ipykernel_2224\3684329844.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])


In [151]:
df_union.query("m2_constr<15")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
3041,3045.00,"Piso de una habitación 11 m², Embajadores-Lavapiés, Madrid",https://www.indomio.es/anuncios/96901373/,NaN,NaN,10.90,11.00,Centro,1,Embajadores,12,Embajadores,None,desconocido,NaN,No indicado,Piso,NaN,NaN,NaN,NaN,1.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,económico,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,119900,Indomio.com,2024-11-20,None,None,28012,None,None,None,No indicado,NaN,"Inmobiliaria en La latina - Puerta de Toledo - El Rastro - Palacio - Madrid - Tecnocasa Consulting Estudio Puerta de Toledo SL¡Encantador piso abuhardillado en el corazón de La Latina!Te ofrecemos la oportunidad de adquirir este precioso piso en una cuarta planta exterior sin ascensor, con el encanto de los techos abuhardillados y lleno de luz natural. Situado en el emblemático barrio de La Latina, este hogar está a solo 3 minutos caminando del famoso Rastro madrileño, que se celebra todos los domingos, y a 5 minutos de las estaciones de metro La Latina y Puerta de Toledo (Línea 5), lo que te permite una excelente conexión con el resto de la ciudad. También estarás a escasos minutos de la Plaza de la Cebada, y a 10 minutos de Madrid Río y del majestuoso Palacio Real, lo que lo convierte en una ubicación privilegiada dentro de la capital.El entorno es inmejorable, con una amplia oferta de bares, restaurantes, mercados y locales de arte, antigüedades y cultura que dotan al barrio de un ambiente vibrante y auténtico, ideal tanto para quienes buscan vivir la esencia de Madrid como para inversores que quieran rentabilizar su compra.La vivienda dispone de un dormitorio independiente, con espacio suficiente para una cama doble y almacenamiento, y un salón amplio y luminoso donde podrás relajarte y disfrutar de la luz natural que entra por las ventanas. La cocina, perfectamente integrada, está diseñada para aprovechar al máximo el espacio, ofreciendo todo lo necesario para el día a día. Además, el baño, de estilo moderno y funcional, está equipado con todas las comodidades.Este piso destaca no solo por su encantadora ubicación, sino también por su carácter singular, ideal para aquellos que buscan un hogar con personalidad en una de las zonas más vibrantes de Madrid. ¡No dejes pasar esta oportunidad de vivir en La Latina, uno de los barrios más auténticos y con más historia de la ciudad!"
3641,NaN,Estudio en venta en Acacias,https://www.pisos.com/comprar/estudio-acacias28005-46671344076_994318/,NaN,NaN,8461.00,13.00,Arganzuela,2,Acacias,22,Acacias (Distrito Arganzuela. Madrid Capital),None,NaN,NaN,NaN,Estudio,NaN,Reformado,1.00,1,1.00,1.00,NaN,NaN,NaN,NaN,1.00,NaN,Eléctrica,Más de 50 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Amueblada y equipada con electrodomésticos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Este,1100

In [152]:
# eliminar https://www.indomio.es/anuncios/96408507/
 #es un edificio


In [153]:
df_union["banos"].value_counts(dropna='False')

banos
1.00     1592
2.00     1531
 1       1084
 2        923
3.00      713
 3+       509
 3        490
4.00      343
5.00      165
6.00       76
7.00       24
8.00       11
10.00       4
9.00        4
11.00       3
12.00       2
13.00       1
Name: count, dtype: int64

In [154]:
df_union.query("banos.isnull()")

,Unnamed: 0,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,antiguedad,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga
2,2.00,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,NaN,NaN,500.00,400.00,Arganzuela,2,Legazpi,24,Legazpi,plaza de italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,<NA>,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,200000,Indomio.com,2024-11-20,None,None,28045,None,None,None,No indicado,1.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio
72,72.00,"Casa unifamiliar Venta, Embajadores-Lavapiés, Madrid",https://www.indomio.es/anuncios/91050501/,NaN,NaN,NaN,NaN,Centro,1,Embajadores,12,Embajadores,None,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,0.00,NaN,NaN,<NA>,NaN,"Individual, frío/calor",Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,550000,Indomio.com,2024-11-20,None,None,28012,None,None,None,No indicado,0.00,"FINCAS FERNANDEZ pone a la Venta de Edificio con Local bar restaurante y 5 estudios con cocina y oficina amueblados, consta de planta baja el restaurante bar equipado con muebles hostelería y cocina equipada y listo para funcionar, da a dos calles. Planta alta hay oficina con buhardilla y 5 estudios con cocina amueblados con electrodomésticos y esta funcionando, esta dando beneficios, tiene 2 sótanos, en uno esta la caldera de gasoil para agua caliente del edificio y otro para el almacén. El restaurante bar y los estudios son independientes, consta de licencias que están activas, mas informacion por telefono. Los mejores inmuebles y el mejor servicio en FINCAS FERNANDEZ, más de 15 años trabajando en el municipio. Si necesita más información puede pasar a visitarnos a Calle SAIZ ROMILLO 11, o ponerse en contacto a través del 916607413"
119,119.00,"Piso de una habitación buen estado, Embajadores-Lavapiés, Madrid",https://www.indomio.es/anuncios/92627639/,NaN,NaN,4.69,32.00,Centro,1,Embajadores,12,Embajadores,None,desconocido,F,No indicado,Piso,NaN,Bueno / Habitable,No,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,"Individual, frío/calor",Sin jardín,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0,0.00,0,0,0.00,0.00,NaN,NaN,NaN,150000,Indomio.com,2024-11-20,None,None,28012,None,None,None,Disponible,0.00,"Se vende piso de parti

In [155]:
df_union["title"]

0                                         Ático Calle de Núñez de Balboa 2, Recoletos, Madrid
1       Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid
2                                         Casa plurifamiliar Plaza de Italia, Legazpi, Madrid
3                Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid
4                Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid
                                                ...                                          
7869                                                               Ático en venta en Vallecas
7870                                                           Ático en venta en Vallehermoso
7871                                                                 Ático en venta en Ventas
7872                                                 Ático en venta en Zona Nueva E Orcasitas
7873                                                        

In [ ]:
df_union["url"].value_counts(dropna=False)

In [156]:
#Guardo la union de los dos datasets
df_union.to_csv("20250302_Union_Sil_Ale.csv",sep=";",index=False)
#df_union.to_feather("20241231_Union_Sil_Ale.ftr")